In [6]:
import pickle
import pandas as pd
import mne
import numpy as np
import re
from datetime import datetime, timezone
import os


In [40]:
# Import the participant's data
eeg_fpath = '/Users/danielgarcia-barnett/Desktop/Coding/cpCST_data_analysis/data/annotated_eeg_data/cpt_data/cpt_data_QCed/cpt_data_extract_07_23_24/sub-M10906060/ses-MOBI1A/eeg_data/sub-M10906060_eeg_cpt.fif'
eeg_obj = mne.io.read_raw(eeg_fpath)

events_fpath = '/Users/danielgarcia-barnett/Desktop/Coding/cpCST_data_analysis/data/annotated_eeg_data/cpt_data/cpt_data_QCed/cpt_data_extract_07_23_24/sub-M10906060/ses-MOBI1A/events_data/sub-M10906060_events_cpt.csv'
events_obj = pd.read_csv(events_fpath)

Opening raw data file /Users/danielgarcia-barnett/Desktop/Coding/cpCST_data_analysis/data/annotated_eeg_data/cpt_data/cpt_data_QCed/cpt_data_extract_07_23_24/sub-M10906060/ses-MOBI1A/eeg_data/sub-M10906060_eeg_cpt.fif...
    Range : 0 ... 613999 =      0.000 ...   613.999 secs
Ready.


/var/folders/w0/4d261sg51qjdphncfst0x4fr0000gn/T/ipykernel_51920/932266107.py:2: RuntimeWarning: This filename (/Users/danielgarcia-barnett/Desktop/Coding/cpCST_data_analysis/data/annotated_eeg_data/cpt_data/cpt_data_QCed/cpt_data_extract_07_23_24/sub-M10906060/ses-MOBI1A/eeg_data/sub-M10906060_eeg_cpt.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_obj = mne.io.read_raw(eeg_fpath)


In [2]:
# Grab a template pkl file to match up the eeg electrode information
dict_fpath_template = '/Users/danielgarcia-barnett/Desktop/third_run/sub-01/ses-01/eeg/sub-01_ses-01_task-checker_run-01_desc-CustomEnvelopes_eeg.pkl'
with open(dict_fpath_template, 'rb') as file:
    dict_template = pickle.load(file)

channel_name_template = dict_template['labels']['channels_info']['channel_name']
anatomy_name_template = dict_template['labels']['channels_info']['anatomy']
laterality_name_template = dict_template['labels']['channels_info']['laterality']

In [143]:
# Utility functions
def is_even(s):
    match = re.search(r'\d+', s)
    if match:
        number = int(match.group())
        return True if number % 2 == 0 else False
    
def remove_numerals(s):
    return s.translate(str.maketrans('', '', '0123456789'))

anatomy_ref_dict = {2: {'Fp': 'frontopolar', 'AF': 'frontal', 'FC':'fronto-central', 'TP':'temporal-parietal', 'CP': 'centro-parietal', 'PO': 'parieto-occipital', 'FT': 'fronto-temporal'}, 
                    1: {'F': 'frontal', 'P': 'parietal', 'I': 'occipital', 'O': 'occipital', 'C': 'central'}}

def anatomy_and_laterality(channel_names):
    anatomy = []
    for ch in channel_names:
        if ch in channel_name_template:
            anatomy.append(anatomy_name_template[channel_name_template.index(ch)])
        elif 'z' in ch:
            anatomy.append(anatomy_ref_dict[len(ch_val)][ch_val])
        else:
            if 'z' in ch:
                ch  = ch.replace('z', '')
            ch_val = remove_numerals(ch)
            anatomy.append(anatomy_ref_dict[len(ch_val)][ch_val])

    laterality = []
    for ch in channel_names:
        if ch in channel_name_template:
            laterality.append(laterality_name_template[channel_name_template.index(ch)])
        else:
            if 'z' in ch:
                laterality.append('midline')
            elif is_even(ch):
                laterality.append('right')
            else:
                laterality.append('left')

    return anatomy, laterality

In [177]:
data_dict = {}

# Adding the eeg data to a dictionary
eeg_time = eeg_obj.times
channel_names = (eeg_obj.info['ch_names'])
anatomy, laterality = anatomy_and_laterality(channel_names)
eeg_indices = list(range(len(channel_names)))
eeg_features = eeg_obj.get_data()
meas_date = eeg_obj.info['meas_date']
ptp_num = 'M109XXXXX'
data_dict['eeg_data'] = {'time': eeg_time, 'labels': {'channels_info': {'index': eeg_indices, 'channels_name': channel_names, 'anatomy': anatomy, 'laterality': laterality}}, 'features': eeg_features, 'features_info': f'EEG data for participant {ptp_num}'}

# Adding the events data to a dictionary
events_time = [round((datetime.fromtimestamp(ts, timezone.utc) - meas_date).total_seconds(), 3) for ts in events_obj.timestamps]
events_indices = list(events_obj.index)
events_labels = events_obj.StimMarkers_alpha.values
events_features = [1 if 'crash' in event.lower() else 0 for event in events_labels]
data_dict['events_data'] = {'time': events_time, 'labels': {'index': events_indices, 'event_labels': events_labels}, 'features': events_features, 'features_info': f'Events data for participant {ptp_num}. Binary classification: 1 = Crash, 0 = Other event marker'}
data_dict

{'eeg_data': {'time': array([0.00000e+00, 1.00000e-03, 2.00000e-03, ..., 6.13997e+02,
         6.13998e+02, 6.13999e+02]),
  'labels': {'channels_info': {'index': [0,
     1,
     2,
     3,
     4,
     5,
     6,
     7,
     8,
     9,
     10,
     11,
     12,
     13,
     14,
     15,
     16,
     17,
     18,
     19,
     20,
     21,
     22,
     23,
     24,
     25,
     26,
     27,
     28,
     29,
     30,
     31,
     32,
     33,
     34,
     35,
     36,
     37,
     38,
     39,
     40,
     41,
     42,
     43,
     44,
     45,
     46,
     47,
     48,
     49,
     50,
     51,
     52,
     53,
     54,
     55,
     56,
     57,
     58,
     59,
     60,
     61,
     62,
     63],
    'channels_name': ['Fp1',
     'Fz',
     'F3',
     'F7',
     'F9',
     'FC5',
     'FC1',
     'C3',
     'T7',
     'CP5',
     'CP1',
     'Pz',
     'P3',
     'P7',
     'P9',
     'O1',
     'Oz',
     'O2',
     'P10',
     'P8',
     'P4',
     'CP2',
     'CP

In [106]:
dict_outpath_template = '/Users/danielgarcia-barnett/Desktop/test.pkl'
with open(dict_outpath_template, 'wb') as file:
    pickle.dump(data_dict, file)

In [4]:
# Grab a template pkl file to match up the eeg electrode information
dict_fpath_template = '/Users/danielgarcia-barnett/Desktop/test.pkl'
with open(dict_fpath_template, 'rb') as file:
    dict_template = pickle.load(file)
dict_template

{'eeg_data': {'time': array([0.00000e+00, 1.00000e-03, 2.00000e-03, ..., 6.13997e+02,
         6.13998e+02, 6.13999e+02]),
  'labels': {'channels_info': {'index': [0,
     1,
     2,
     3,
     4,
     5,
     6,
     7,
     8,
     9,
     10,
     11,
     12,
     13,
     14,
     15,
     16,
     17,
     18,
     19,
     20,
     21,
     22,
     23,
     24,
     25,
     26,
     27,
     28,
     29,
     30,
     31,
     32,
     33,
     34,
     35,
     36,
     37,
     38,
     39,
     40,
     41,
     42,
     43,
     44,
     45,
     46,
     47,
     48,
     49,
     50,
     51,
     52,
     53,
     54,
     55,
     56,
     57,
     58,
     59,
     60,
     61,
     62,
     63],
    'channels_name': ['Fp1',
     'Fz',
     'F3',
     'F7',
     'F9',
     'FC5',
     'FC1',
     'C3',
     'T7',
     'CP5',
     'CP1',
     'Pz',
     'P3',
     'P7',
     'P9',
     'O1',
     'Oz',
     'O2',
     'P10',
     'P8',
     'P4',
     'CP2',
     'CP

In [18]:
july_22_extract = '/Users/danielgarcia-barnett/Desktop/Coding/cpCST_data_analysis/data/annotated_eeg_data/cpt_data/cpt_data_QCed/cpt_data_extract_07_22_24'
july_23_extract = '/Users/danielgarcia-barnett/Desktop/Coding/cpCST_data_analysis/data/annotated_eeg_data/cpt_data/cpt_data_QCed/cpt_data_extract_07_23_24'
previous_extract_list = os.listdir(july_22_extract) + os.listdir(july_23_extract)

In [16]:
cpt_dict_dirs = '/Users/danielgarcia-barnett/Desktop/cpt_dict'
dict_extract_list = [sub.split('_')[0] for sub in os.listdir(cpt_dict_dirs)]

In [19]:
for ptp in dict_extract_list:
    if ptp not in previous_extract_list:
        print(f'Participant {ptp} has not been extracted.')

Participant sub-M10963307 has not been extracted.
Participant sub-M10987516 has not been extracted.


In [20]:
with open('/Users/danielgarcia-barnett/Desktop/cpt_dict/sub-M10901084_cpt.pkl', 'rb') as f:
    test_dict = pickle.load(f)

In [23]:
test_dict['eeg_data'].keys()

dict_keys(['time', 'labels', 'features', 'features_info'])